In [97]:
import numpy as np
import pandas as pd
import gc
gc.collect()



path = './data/'
player_ext = 'Players_2019.csv'
event_ext = 'Events_2019.csv'
teams_ext = 'Teams.csv'

events = pd.read_csv(path+event_ext)
players = pd.read_csv(path+player_ext)

In [98]:
players.head()

,PlayerID,Season,TeamID,PlayerName
0,653508,2019,1101,BJ_MAXWELL
1,653509,2019,1101,CHASE_COBB
2,653510,2019,1101,CLAY_GAYMAN
3,653511,2019,1101,DAMIEN_DANIELS
4,653512,2019,1101,HAYDEN_FARQUHAR


In [99]:
events.head()

,EventID,Season,DayNum,WTeamID,LTeamID,WPoints,LPoints,ElapsedSeconds,EventTeamID,EventPlayerID,EventType
0,30099809,2019,1,1104,1380,0,0,18,1380,657441,assist
1,30099810,2019,1,1104,1380,0,2,18,1380,657447,made2_jump
2,30099811,2019,1,1104,1380,2,2,42,1104,653561,made2_jump
3,30099812,2019,1,1104,1380,0,0,58,1380,657437,turnover
4,30099813,2019,1,1104,1380,0,0,63,1104,653565,assist


In [100]:
len(events)

2543316

In [101]:
list(events)

['EventID',
 'Season',
 'DayNum',
 'WTeamID',
 'LTeamID',
 'WPoints',
 'LPoints',
 'ElapsedSeconds',
 'EventTeamID',
 'EventPlayerID',
 'EventType']

### Validation

In [102]:
# beautiful
events.isna().sum()

EventID           0
Season            0
DayNum            0
WTeamID           0
LTeamID           0
WPoints           0
LPoints           0
ElapsedSeconds    0
EventTeamID       0
EventPlayerID     0
EventType         0
dtype: int64

### Feature Engineering

To add:
- game id
- current OT

Need: 
- possession time
- time remaining in game (including factoring OT)
- change of possession indicator

In [103]:
# load compact results
# cr_path = './data/RegularSeasonCompactResults.csv'
# cr = pd.read_csv(cr_path)

# cr_path = './data/Prelim2019_RegularSeasonCompactResults.csv'
# cr = pd.read_csv(cr_path)



In [104]:
events['GameID'] = events['Season'].astype(str) + events['DayNum'].astype(str) + events['WTeamID'].astype(str) + events['LTeamID'].astype(str)
events['OT'] = (events['ElapsedSeconds'] - 2400)/300
events['OT'] = np.where(events['OT'] < 0, 0, np.ceil(events['OT'])).astype(int)


In [105]:
events['TimeRemain'] = 2400 + (events['OT'] * 300) - events['ElapsedSeconds']


In [106]:
print(events.head())

    EventID  Season  DayNum  WTeamID  LTeamID  WPoints  LPoints  \
0  30099809    2019       1     1104     1380        0        0   
1  30099810    2019       1     1104     1380        0        2   
2  30099811    2019       1     1104     1380        2        2   
3  30099812    2019       1     1104     1380        0        0   
4  30099813    2019       1     1104     1380        0        0   

   ElapsedSeconds  EventTeamID  EventPlayerID   EventType         GameID  OT  \
0              18         1380         657441      assist  2019111041380   0   
1              18         1380         657447  made2_jump  2019111041380   0   
2              42         1104         653561  made2_jump  2019111041380   0   
3              58         1380         657437    turnover  2019111041380   0   
4              63         1104         653565      assist  2019111041380   0   

   TimeRemain  
0        2382  
1        2382  
2        2358  
3        2342  
4        2337  


In [107]:
print(list(events))

['EventID', 'Season', 'DayNum', 'WTeamID', 'LTeamID', 'WPoints', 'LPoints', 'ElapsedSeconds', 'EventTeamID', 'EventPlayerID', 'EventType', 'GameID', 'OT', 'TimeRemain']


In [108]:
# let's calculate usage lost going into 2019

# usage calc:
# 100*((Player’s Field Goal Attempts)+0.44*(Player’s Free Throw Attempts)+(Player’s Turnovers))*(Team’s Total Minutes)
# /
# ((Team’s Total Field Goal Attempts)+0.44*(Team’s Total Free Throw Attempts)+Team’s Total Turnovers))*5*(Player’s Minutes)

# need field goal attempts, free throw attempts, turnovers
usage_events = ['made1_free','made2_dunk','made2_jump','made2_lay','made2_tip','made3_jump',
                'miss1_free','miss2_dunk','miss2_jump','miss2_lay','miss2_tip','miss3_jump',
               'turnover']

usage = events.loc[events['EventType'].isin(usage_events)]

fgas = ['made2_dunk','made2_jump','made2_lay','made2_tip','made3_jump','miss2_dunk','miss2_jump','miss2_lay','miss2_tip','miss3_jump']
ftas = ['made1_free','miss1_free']

usage.loc[usage['EventType'].isin(fgas), 'EventType'] = 'FGA'
usage.loc[usage['EventType'].isin(ftas), 'EventType'] = 'FTA'


pgb = usage.groupby(['EventPlayerID','EventType'])['EventID'].count().unstack(level=-1).reset_index()
pgb = pgb.rename(columns={'EventPlayerID':'PlayerID'})

# pgb = pgb.drop(columns=['EventType'])

tgb = usage.groupby(['EventTeamID','EventType'])['EventID'].count().unstack(level=-1).reset_index()

# tgb = tgb.drop(columns=['EventType'])

tgb = tgb.rename(columns={'FGA':'TFGA','FTA':'TFTA','turnover':'TTO','EventTeamID':'TeamID'})
tgb.head()


EventType,TeamID,TFGA,TFTA,TTO
0,1101,1602,553,338
1,1102,1715,485,409
2,1103,1825,505,369
3,1104,1876,722,449
4,1105,1738,438,477


In [109]:
players = players.drop(columns=['Season'])
pgb = pd.merge(left=pgb, right=players, how='left', on=['PlayerID','PlayerID'])
pgb = pd.merge(left=pgb, right=tgb, how='left', on=['TeamID','TeamID'])
pgb.head()

,PlayerID,FGA,FTA,turnover,TeamID,PlayerName,TFGA,TFTA,TTO
0,653508,140.0,29.0,15.0,1101,BJ_MAXWELL,1602,553,338
1,653509,3.0,2.0,1.0,1101,CHASE_COBB,1602,553,338
2,653510,26.0,10.0,11.0,1101,CLAY_GAYMAN,1602,553,338
3,653511,95.0,22.0,24.0,1101,DAMIEN_DANIELS,1602,553,338
4,653512,137.0,31.0,20.0,1101,HAYDEN_FARQUHAR,1602,553,338


In [110]:
pgb['Usage'] = (pgb['FGA'] + 0.44*pgb['FTA'] + pgb['turnover'])/(pgb['TFGA']+0.44*pgb['TFTA']+pgb['TTO'])
pgb.head()

,PlayerID,FGA,FTA,turnover,TeamID,PlayerName,TFGA,TFTA,TTO,Usage
0,653508,140.0,29.0,15.0,1101,BJ_MAXWELL,1602,553,338,0.076837
1,653509,3.0,2.0,1.0,1101,CHASE_COBB,1602,553,338,0.002235
2,653510,26.0,10.0,11.0,1101,CLAY_GAYMAN,1602,553,338,0.018962
3,653511,95.0,22.0,24.0,1101,DAMIEN_DANIELS,1602,553,338,0.058938
4,653512,137.0,31.0,20.0,1101,HAYDEN_FARQUHAR,1602,553,338,0.078156


In [111]:
use = pgb[['TeamID','PlayerID','PlayerName','Usage']]

teams_ext = 'Teams.csv'
teams = pd.read_csv(path+teams_ext)
teams = teams[['TeamID','TeamName']]

use = pd.merge(use, teams, how='left',on=['TeamID','TeamID'])

use = use[['TeamName','PlayerName','Usage']]
use.head()


,TeamName,PlayerName,Usage
0,Abilene Chr,BJ_MAXWELL,0.076837
1,Abilene Chr,CHASE_COBB,0.002235
2,Abilene Chr,CLAY_GAYMAN,0.018962
3,Abilene Chr,DAMIEN_DANIELS,0.058938
4,Abilene Chr,HAYDEN_FARQUHAR,0.078156


In [112]:
# print(use.TeamName.unique())

In [113]:
needed_teams = ['Clemson','Virginia Tech','Duke','Kansas','Fresno St','Oregon','Georgia Tech','NC State',
                'Michigan St','Kentucky','Nevada','Utah',"St Mary's CA",'Wisconsin']
use = use.loc[use.TeamName.isin(needed_teams)]
print(len(use.TeamName.unique()))


14


In [114]:
use.to_csv('2019_usage.csv',index=None)

In [115]:
rosters = pd.read_csv('2019_rosters.csv')
rosters = rosters.drop(columns=['Unnamed: 0','number'])

rosters.head()

,name,position,height,weight,class,hometown,Team
0,Clyde Trapp,G,"6' 4""",194 lbs,JR,"Eastover, SC",Clemson
1,Jonathan Baehre,F,"6' 10""",209 lbs,JR,"Wiesbaden, Germany",Clemson
2,Al-Amir Dawes,G,"6' 2""",180 lbs,FR,"Newark, NJ",Clemson
3,Chase Hunter,G,"6' 3""",200 lbs,FR,"Atlanta, GA",Clemson
4,Nick Honor,G,"5' 10""",205 lbs,SO,"Orlando, FL",Clemson


In [116]:
use.loc[:,'PlayerName'] = use['PlayerName'].str.replace("_"," ")
use.loc[:,'PlayerName'] = use['PlayerName'].str.title()

use.head()

,TeamName,PlayerName,Usage
683,Clemson,Aamir Simms,0.117890
684,Clemson,Clyde Trapp,0.096289
685,Clemson,David Skara,0.095564
686,Clemson,Elijah Thomas,0.170467
687,Clemson,Hunter Tyson,0.028818


In [117]:
roster_to_use = {
    "Alex O'Connell":'Alex Oconnell',
    "C.J. Bryce":'Cj Bryce',
    "D'Mitrik Trice":"Dmitrik Trice",
    "DJ Funderburk":'Derek Funderburk',
    "David McCormack":'David Mccormack',
    "EJ Montgomery":'Ej Montgomery',
    "James Banks III":'James Banks Iii',
    "John Newman III":'John Newman Iii',
    "Marcus Bingham Jr.":'Marcus Bingham Jr',
    "P.J. Horne":'Pj Horne',
    "Walt McGrory":'Walt Mcgrory'
}

use_to_roster = dict(map(reversed, roster_to_use.items()))

In [118]:
use = use.replace({"PlayerName": use_to_roster})

In [120]:
### check for similar names to match 
roster_names = list(rosters.name.values)
use_names = list(use.PlayerName.values)

print("Use minus roster")
udiff = list(set(use_names)-set(roster_names))
udiff.sort()
print(udiff)

print("Roster minus use")
rdiff = list(set(roster_names)-set(use_names))
rdiff.sort()
print(rdiff)

import difflib

for rword in rdiff:
    print(rword, difflib.get_close_matches(rword,udiff))




Use minus roster
['Abdoulaye Gueye', 'Abu Kigab', 'Ahmed Hill', 'Alex Mudronja', 'Antonio Vrankovic', 'Beau Rydalch', 'Bol Bol', 'Brandon Alston', 'Brandon Morley', 'Braxton Huggins', 'Brennan Besser', 'Caleb Martin', 'Cam Reddish', 'Charles Jones', 'Charles Thomas Iv', 'Charlie Moore', 'Cody Martin', 'Corey Henson', 'Curtis Haywood Ii', 'D T', 'David Cunningham', 'David Skara', 'Dedric Lawson', 'Deshon Taylor', 'Donnie Tillman', 'Ehab Amin', 'Eric Lockett', 'Ethan Happ', 'Evan Jester', 'Garrett Luinstra', 'Ian Steere', 'Javan White', 'Javin Delaurier', 'Jayce Johnson', 'Jemarl Baker Jr', 'Johnny Mcwilliams', 'Jonny David', 'Jordan Brown', 'Jordan Caroline', 'Jordan Hunter', 'Keldon Johnson', 'Kenny Goins', 'Kenny Wooten', 'Kerry Blackshear', 'Khalil Iverson', 'Kj Lawson', 'Kyle Clark', 'Lagerald Vick', 'Louis King', 'Lyles Davis', 'Marcquise Reed', 'Marques Bolden', 'Matt Mcquaid', 'Miles Norris', 'Nick Ward', 'Nickeil Alexander-Walker', 'Novak Topalovic', 'Parker Van Dyke', 'Paul Whi

In [124]:
use['Graduated'] = np.where(use['PlayerName'].isin(roster_names),"Still On Roster","Lost")


In [128]:
ugb = use.groupby(['TeamName','Graduated'])['Usage'].sum().unstack(-1).reset_index()
ugb.sort_values(by="Still On Roster", ascending=False)

Graduated,TeamName,Lost,Still On Roster
10,St Mary's CA,0.115301,0.874914
7,NC State,0.269264,0.728229
3,Georgia Tech,0.265919,0.723850
6,Michigan St,0.379882,0.612745
13,Wisconsin,0.392056,0.591931
2,Fresno St,0.533454,0.462032
0,Clemson,0.536216,0.453075
4,Kansas,0.567106,0.427084
9,Oregon,0.669019,0.323424
11,Utah,0.664652,0.320906


In [132]:
returning = use.loc[use.Graduated=="Still On Roster"]
returning = returning.drop(columns=['Graduated'])
ugb = ugb.rename(columns={'Lost':'Available Usage'})
ugb = ugb.drop(columns=['Still On Roster'])
returning = pd.merge(returning, ugb, how='left',on=['TeamName','TeamName'])
returning = returning.sort_values(by=['Usage'],ascending=False)
print(returning.head(50))

         TeamName         PlayerName     Usage  Available Usage
71   St Mary's CA        Jordan Ford  0.263382         0.115301
43    Michigan St    Cassius Winston  0.225073         0.379882
72   St Mary's CA        Malik Fitts  0.195775         0.115301
22   Georgia Tech      Jose Alvarado  0.183903         0.265919
64         Oregon   Payton Pritchard  0.174513         0.669019
2         Clemson      Elijah Thomas  0.170467         0.536216
90      Wisconsin     D'Mitrik Trice  0.167600         0.392056
31         Kansas       Devon Dotson  0.154185         0.567106
88      Wisconsin       Brad Davison  0.150170         0.392056
26   Georgia Tech      Michael Devoe  0.149244         0.265919
17      Fresno St        Nate Grimes  0.142974         0.533454
21   Georgia Tech    James Banks III  0.141595         0.265919
81           Utah        Timmy Allen  0.139989         0.664652
55       NC State         C.J. Bryce  0.134647         0.269264
57       NC State      Devon Daniels  0.

In [9]:
events.groupby(['EventType'])['EventID'].count()


EventType
assist        143827
block          35855
foul_pers     195974
foul_tech        727
made1_free    146907
made2_dunk     19173
made2_jump     58823
made2_lay     112392
made2_tip       2643
made3_jump     83688
miss1_free     61075
miss2_dunk      2517
miss2_jump    102592
miss2_lay      86044
miss2_tip       1144
miss3_jump    159586
reb_dead       30928
reb_def       273589
reb_off       108474
steal          67640
sub_in        327515
sub_out       326809
timeout        29704
timeout_tv     24098
turnover      141592
Name: EventID, dtype: int64

In [10]:
sample = events.loc[events['GameID'] == '201913214631217']
sample.to_csv('./output/sample_game.csv',index=False)



Change of possession calculation is complicated. It's needed to know how each possession ends (did the team make one free throw or two)?

There are some event types that may or may not end the possession, like a made free throw.

I plan to resolve some of these in different ways.
If a team misses their last free throw, then there should be a rebound coupled with it.
Therefore if there is no rebound event coupled with it, they must have made their last free throw (unless on an air ball? but that's an edge case i'll ignore).

In [13]:
all_events = list(events.EventType.unique())

# offense at time of event
off_events = ['assist','made2_jump','turnover','made3_jump','miss2_jump','miss3_jump','reb_off','miss1_free',
             'made1_free','made2_dunk','made2_lay','miss2_lay','reb_dead','miss2_dunk','made2_tip','miss2_tip']

def_events = ['block','steal','reb_def']

neu_events = ['timeout_tv','foul_tech','foul_pers','timeout','sub_out','sub_in']

events['EventTeam'] = np.where(events['EventTeamID']==events['WTeamID'],events['WTeamID'],events['LTeamID'])
events['NonEventTeam'] = np.where(events['EventTeamID']==events['WTeamID'],events['LTeamID'],events['WTeamID'])


In [ ]:
# create column of last event team for neutral event situations
events['PossTeam'] = np.where(events['EventType'].isin(off_events), events['EventTeamID'], )
events['PossTeam'] = np.where(events['EventType'].isin(def_events), events['NonEventTeam'], events['PossTeam'])


events['PossTeam'] = events['PossTeam'].copy().astype(int)
# mark neutral events as non change of possession -- to my knowledge these are accompanied by events that do mark a turnover
# events['PossTeam'] = events['PossTeam'].fillna(method='ffill')

When there's a difference in poss team from the previous row, I'm going to mark a change of possession (COP).

One issue is that multiple events can happen within the same second (missed FT, defensive reb). So I need to sort by time remaining then Def Team to help with this, so there's only a max of one change of possession per second.

There seem to be a couple of cases where this is insufficient. For example, a late defensive rebound, foul on rebounding player, missed free throw, defensive rebound can all happen happen in a second. The only fix for this right now is that I'm going to mark all defensive rebounds as a COP. This fix probably has holes in it, and I'll be more thorough in the future.

In [14]:
events = events.sort_values(by=['GameID','OT','TimeRemain','PossTeam'],ascending=[True,True,False,True])
events.head()

,EventID,Season,DayNum,WTeamID,LTeamID,WPoints,LPoints,ElapsedSeconds,EventTeamID,EventPlayerID,EventType,GameID,OT,TimeRemain,EventTeam,OtherTeam,PossTeam
1896418,31996227,2019,100,1101,1311,0,0,29,1101,653515,assist,201910011011311,0,2371,1101,1311,1101.0
1896419,31996228,2019,100,1101,1311,3,0,29,1101,653508,made3_jump,201910011011311,0,2371,1101,1311,1101.0
1896420,31996229,2019,100,1101,1311,0,0,58,1311,656456,turnover,201910011011311,0,2342,1311,1101,1311.0
1896421,31996230,2019,100,1101,1311,0,0,60,1101,653519,steal,201910011011311,0,2340,1101,1311,1311.0
1896422,31996231,2019,100,1101,1311,0,0,63,1311,656456,foul_pers,201910011011311,0,2337,1311,1101,1311.0


In [15]:
gb = events.groupby(['GameID'])['PossTeam'].diff().reset_index()
# nans for first event of the game
gb = gb.fillna(0)
# change of possession
gb['COP'] = np.where(gb['PossTeam']!=0, 1, 0)

events['COP'] = gb['COP'].copy()

# for multiple COP in same second
events['COP'] = np.where(events['EventType']=='reb_def', 1, events['COP'].copy())

events.head(25)

,EventID,Season,DayNum,WTeamID,LTeamID,WPoints,LPoints,ElapsedSeconds,EventTeamID,EventPlayerID,EventType,GameID,OT,TimeRemain,EventTeam,OtherTeam,PossTeam,COP
1896418,31996227,2019,100,1101,1311,0,0,29,1101,653515,assist,201910011011311,0,2371,1101,1311,1101.0,1
1896419,31996228,2019,100,1101,1311,3,0,29,1101,653508,made3_jump,201910011011311,0,2371,1101,1311,1101.0,1
1896420,31996229,2019,100,1101,1311,0,0,58,1311,656456,turnover,201910011011311,0,2342,1311,1101,1311.0,0
1896421,31996230,2019,100,1101,1311,0,0,60,1101,653519,steal,201910011011311,0,2340,1101,1311,1311.0,1
1896422,31996231,2019,100,1101,1311,0,0,63,1311,656456,foul_pers,201910011011311,0,2337,1311,1101,1311.0,0
1896423,31996232,2019,100,1101,1311,0,0,68,1101,653515,assist,201910011011311,0,2332,1101,1311,1101.0,0
1896424,31996233,2019,100,1101,1311,6,0,68,1101,653519,made3_jump,201910011011311,0,2332,1101,1311,1101.0,0
1896425,31996234,2019,100,1101,1311,0,0,92,1101,653508,reb_def,201910011011311,0,2308,1101,1311,1311.0,1
1896426,31996235,2019,100,1101,1311,0,0,92,1311,656457,miss3_jump,201910011011311,0,2308,1311,1101,1311.0,0
1896427,31996236,2019,100,1101,1311,0,0,107,1101,653519,miss3_jump,201910011011311,0,2293,1101,1311,1101.0,1


In [16]:
sample = events.loc[events['GameID'] == '2019111041380']
sample.to_csv('./output/sample_game.csv',index=False)

